In [2]:
from azureml.core import Workspace, Experiment, Dataset, Datastore, Run, Model
from azureml.exceptions import UserErrorException
from sklearn.model_selection import train_test_split

In [4]:
ws = Workspace.from_config()

Performing interactive authentication. Please follow the instructions on the terminal.


The default web browser has been opened at https://login.microsoftonline.com/organizations/oauth2/v2.0/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.


Interactive authentication successfully completed.


In [1]:
import pandas as pd

In [6]:
df = pd.read_csv("../DiamondData.csv")
df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,1.50,Fair,G,SI1,64.5,57.0,10352.0,7.15,7.09,4.59
1,0.70,Ideal,E,VS2,61.4,57.0,2274.0,5.72,5.78,3.53
2,1.22,Premium,G,VS1,61.3,58.0,8779.0,6.91,6.89,4.23
3,0.51,Premium,E,VS2,62.5,60.0,1590.0,5.08,5.10,3.18
4,2.02,Very Good,J,SI2,59.2,60.0,11757.0,8.27,8.39,4.91


In [7]:
try:
    dataset = Dataset.get_by_name(ws, name="my-dimond-dataset")
except UserErrorException as ue:
    datastore = Datastore.get_default(ws)
    dataset = Dataset.Tabular.register_pandas_dataframe(
        df,
        datastore,
        show_progress=True,
        name='my-dimond-dataset',
        description='This is some dataset'
    )

In [24]:
df = dataset.to_pandas_dataframe()
df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,1.50,Fair,G,SI1,64.5,57.0,10352.0,7.15,7.09,4.59
1,0.70,Ideal,E,VS2,61.4,57.0,2274.0,5.72,5.78,3.53
2,1.22,Premium,G,VS1,61.3,58.0,8779.0,6.91,6.89,4.23
3,0.51,Premium,E,VS2,62.5,60.0,1590.0,5.08,5.10,3.18
4,2.02,Very Good,J,SI2,59.2,60.0,11757.0,8.27,8.39,4.91


In [25]:
df = df[["price","carat","depth","x","y","z"]].fillna(0)


In [26]:
y = df["price"]
X = df[["carat","depth","x","y","z"]]

In [13]:
from tensorflow.keras.models import load_model

# Ladda in förtränad model
CNN_model_pretrain = load_model('../model/MNIST_CNN_V1_100_epochs.h5')

# Ladda in historiken för förtränad model

with open('../model/MNIST_CNN_V1_100_epochs_hist', 'rb') as f:
    # load using pickle de-serializer
    history_pretrain = pickle.load(f)

,carat,depth,x,y,z
0,1.50,64.5,7.15,7.09,4.59
1,0.70,61.4,5.72,5.78,3.53
2,1.22,61.3,6.91,6.89,4.23
3,0.51,62.5,5.08,5.10,3.18
4,2.02,59.2,8.27,8.39,4.91
...,...,...,...,...,...
49995,2.01,62.4,8.01,7.99,4.99
49996,0.80,62.6,5.93,5.89,3.70
49997,0.90,61.4,6.11,6.20,3.78
49998,1.24,61.4,6.85,6.90,4.22


In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [28]:
from sklearn.linear_model import LogisticRegression

In [34]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)

KeyboardInterrupt: 

In [38]:
from dataclasses import dataclass